# Lightning map

This Jupyter notebook uses data from [Meteorologisk institutt](https://frost.met.no/api.html#!/lightning/getLightning "met.no") to create an interactive map of recorded lightnings. After clicking on the map, statistics on lightnings per year for the clicked location can be found in a bar chart and table next to the map.

This map uses data from the Norwegian mainland only. Follow the link above to access the data or to retrieve data from offshore locations and other Nordic countries.

Libraries

In [37]:
import pandas as pd
import holoviews as hv
import panel as pn
import numpy as np
import hvplot.pandas
import pyproj

from holoviews import opts
from holoviews.operation.datashader import datashade
from holoviews.element.tiles import CartoDark
from bokeh.models import HoverTool
from collections import Counter

hv.extension('bokeh', logo = False)

Data
- Lightning locations in Web Mercator coordinates, stored as csv file


In [20]:
data = pd.read_csv('data/lyn_webmerc.csv')

Reference areas

1000^2 = 1 km2 </br>
10000^2 = 100 km2

In [21]:
area_small = 1000**2
area_large = 10000**2

Plot sizing options

In [22]:
map_opts = dict(xaxis = None, yaxis = None, min_height = 550, min_width = 600,
                responsive = True, padding = 0, active_tools = ['wheel_zoom'])
chart_opts = dict(height = 250, width = 800, responsive = False, padding = 0)
table_opts = dict(height = 100, width = 800, padding = 0, fit_columns = True)

Table style options

In [23]:
def hide_index(plot, element):
    plot.handles['table'].index_position = None

Bar chart hover and tooltips

In [24]:
bar_tooltips = [
    ('År', '@year'),
    ('Lyntetthet', '@dist{0.}')
    ]
bar_hover = HoverTool(tooltips = bar_tooltips)

Calculate radius from circle area

In [25]:
def calc_radius(area):
    return np.sqrt(area / np.pi)

Distance correction <br>
- See [gis.stackexchange.com](https://gis.stackexchange.com/questions/14528/better-distance-measurements-in-web-mercator-projection) for explanation

In [26]:
e = 0.081819191

def adj_x(latitude):
    lat = np.radians(latitude) 
    return np.cos(lat) / np.sqrt(1 - e**2 * np.sin(lat)**2)

def adj_y(latitude):
    lat = np.radians(latitude)
    return np.cos(lat) * (1 - e**2) / (1 - e**2 * np.sin(lat)**2)**(3/2)

Coordinate transformation: EPSG:3857 --> EPSG:4326

In [27]:
p = pyproj.Transformer.from_crs('EPSG:3857', 'EPSG:4326')

def transform_lat(x_coord, y_coord):
    lat, lon = p.transform(x_coord, y_coord)
    return lat

Count points within a distance (radius) from a given location (x, y)
- In this context: count the number of lightnings in a 1 km2 & 100 km2 large area around location x, y

In [28]:
def points_within(radius, x, y):
    y_transf = transform_lat(x, y)
    x_factor, y_factor = adj_x(y_transf), adj_y(y_transf)
    dist_x, dist_y = np.absolute(data['x'] - x) * x_factor, np.absolute(data['y'] - y) * y_factor
    dist = np.sqrt(np.square(dist_x) + np.square(dist_y))
    return [(x0, y0, year) for x0, y0, year, d in zip(data['x'], data['y'], data['year'], dist) if d <= radius]

Series from points
- Count lightnings per year

In [29]:
min_year, max_year = data['year'].min(), data['year'].max() + 1
year_range = range(min_year, max_year)
standard_dict = dict(zip(year_range, [0] * len(year_range)))
    
def points_series(pts):
    years = [point[2] for point in pts]
    occ = dict(Counter(years))
    occ_dict = {**standard_dict, **occ}
    return pd.Series(occ_dict).rename_axis('year').astype(int).rename('dist')

Bar chart

In [30]:
def create_bars(series):
    bars = series.hvplot.bar(
        'year', alpha = 0.65, xlabel = 'År',
        ylabel = 'Lyntetthet [lyn / (100 km2 * år)]',
        title = 'Lyntetthet [lyn / (100 km2 * år)]',
        color = 'black', tools = [bar_hover]
    ).options(ylim = (0, series.max()), **chart_opts)
    return bars

Table

In [31]:
def create_table(s1, s2):
    df = pd.DataFrame({'Area100': s2, 'Area1': s1}).transpose()
    df.insert(loc = 0, column = 'km2', value = [100, 1])
    return df.hvplot.table(
        sortable = True, selectable = False
    ).opts(**table_opts, hooks = [hide_index])

Map

In [32]:
points = hv.Points(data, ['x', 'y'])
stream = hv.streams.Tap(source = points, x = 1189390, y = 8378600)
data_opac = pn.widgets.FloatSlider(
        value = 0.90, name = 'Ugjennomskinlighet lyndata',
        start = 0.00, end = 1.00, sizing_mode = 'stretch_width'
    )

@pn.depends(stream.param.x, stream.param.y)
def draw_point(x, y):
    return hv.Points((x, y)).opts(marker = 'x', color = 'white', size = 18)

@pn.depends(stream.param.x, stream.param.y)
def draw_ring1(x, y):
    return hv.Ellipse(
        x, y,
        2 * calc_radius(area_small) / adj_y(transform_lat(x, y))
    ).opts(color = 'lightblue', alpha = 0.6)

@pn.depends(stream.param.x, stream.param.y)
def draw_ring2(x, y):
    return hv.Ellipse(
        x, y,
        2 * calc_radius(area_large) / adj_y(transform_lat(x, y))
    ).opts(color = 'lightblue', alpha = 0.6)

@pn.depends(x = stream.param.x, y = stream.param.y, alpha = data_opac.param.value)
def draw_lightnings(x, y, alpha):
    factor = 1.5
    pts = points_within(calc_radius(area_large), x, y)
    coords = [(i[0], i[1]) for i in pts]
    return hv.Points(coords).opts(color = 'lightblue', size = 1, fill_alpha = min(factor * alpha, 1), line_alpha = min(factor * alpha, 1))

def create_map():
    lightnings = (CartoDark().opts(**map_opts) *
                 datashade(points, cmap = 'Plasma').apply.opts(alpha = data_opac) *
                 hv.DynamicMap(draw_point) *
                 hv.DynamicMap(draw_ring1) *
                 hv.DynamicMap(draw_ring2) *
                 hv.DynamicMap(draw_lightnings)) 
    return pn.Column(data_opac, lightnings)

Bar chart + table

In [33]:
@pn.depends(stream.param.x, stream.param.y)
def create_timeseries(x, y):
    series_large = points_series(points_within(calc_radius(area_large), x, y))
    series_small = points_series(points_within(calc_radius(area_small), x, y))
    bars = create_bars(series_large)
    table = create_table(series_small, series_large)
    text = 'Tabell: Lyn på valgt adresse pr år innenfor 100 km2 / 1 km2'
    return pn.Column(bars, '', text, table)

App

In [36]:
pn.Column(
    '# Lyntetthet - klikk på kartet for statistikk',
    pn.Row(create_map, create_timeseries),
    'Data: Meteorologisk institutt, applikasjon: TS, 2021',
    width_policy = 'max', height_policy = 'max'
).servable()

Column(height_policy='max', width_policy='max')
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] Column
                    [0] FloatSlider(name='Ugjennomskinlighet l..., sizing_mode='stretch_width', value=0.9)
                    [1] HoloViews(DynamicMap)
        [1] ParamFunction(function)
    [2] Markdown(str)